# JSSP comparação entre euristicas Classicas: tratamento dos dados 

In [1]:
# %pip install pandas
# %pip install seaborn
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:

csv_filename = "simulated_annealing_results.csv"
df = pd.read_csv(csv_filename)
print("\nPrimeiras 5 linhas do CSV:")
print(df.head())

print(f"\nEstatísticas dos tempos de execução:")
print(f"Tempo médio: {df['execution_time'].mean():.4f}s")
print(f"Tempo mínimo: {df['execution_time'].min():.4f}s")
print(f"Tempo máximo: {df['execution_time'].max():.4f}s")

print(f"\nEstatísticas do fitness:")
print(f"Fitness médio: {df['fitness'].mean():.4f}")
print(f"Melhor fitness: {df['fitness'].min():.4f}")
print(f"Pior fitness: {df['fitness'].max():.4f}")


Primeiras 5 linhas do CSV:
     id  execution_time  fitness                             solution_vector
0   509        0.025012      1.0    [0.6966838242420641, 0.5067566407361701]
1  1018        0.026170      1.0  [0.9273990581711936, 0.002496059185002286]
2  1527        0.064660      1.0  [0.20803180807228905, 0.18125035538485856]
3  2036        0.016640      1.0     [0.949709719164603, 0.3951293248385438]
4  2545        0.016053      1.0    [0.2580897276104821, 0.9071301602386016]

Estatísticas dos tempos de execução:
Tempo médio: 0.0189s
Tempo mínimo: 0.0149s
Tempo máximo: 0.0647s

Estatísticas do fitness:
Fitness médio: 1.0000
Melhor fitness: 1.0000
Pior fitness: 1.0000
